In [2]:
%cd ".."

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import tape_data
import munsell_data
import optimization as opt
from experiment import *
from polarization import phase_shift, transmission_mueller

/Users/yuta-fu/naist


In [21]:
# Params 
n_coreset = 10     # Number of filters to choose
n_basis = 2        # Number of basis function to describe transmission spectra of filters
n_samples = 5000   # Number of randomly generated filters to choose from

# Sample theta randomly (n_meas = 0) or use preset list of thetas to generate samples
# thetas = np.linspace(0.0, 180.0, 10)
# n_meas = len(thetas)
n_meas = 0

wp_hd = Waveplate(tape_data.THICKNESS_HD, tape_data.BIREFRINGENCE_HD)
wp_ss = Waveplate(tape_data.THICKNESS_SS, tape_data.BIREFRINGENCE_SS)

In [22]:
wavelengths = munsell_data.MUNSELL_CHIPS_WAVELENGTHS

print("Creating dataset...", end = "")
dataset = opt.create_dataset(wavelengths, [wp_hd, wp_ss], n_samples = n_samples, thetas = None, fixed_waveplates = True)
print(dataset.shape)
print(" done.")

dataset_transmissions = np.array([config.transmission(wavelengths) for config in dataset.flatten()])

print("Choosing optimal filters...", end = "")
opt_filters, pca_basis = opt.choose_optimal_filters(dataset, wavelengths, n_coreset = n_coreset, n_meas = n_meas)
print(" done.")
opt_transmissions = np.array([config.transmission(wavelengths) for config in opt_filters])

# Project all filter transmissions down to basis function coefficients
filter_coefs, _, _, _ = np.linalg.lstsq(pca_basis.transpose(), dataset_transmissions.transpose(), rcond=None)
print(filter_coefs.shape)

# Project optimal (coreset) filter transmissions down to basis function coefficients
coreset, _, _, _ = np.linalg.lstsq(pca_basis.transpose(), opt_transmissions.transpose(), rcond=None)
print(coreset.shape)

Creating dataset...(5000,)
 done.
Choosing optimal filters... done.
(4, 5000)
(4, 10)


In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plots = []
colors = ['Tealgrn', 'Purp', 'Oryel']

if n_basis == 2:
    scatter_all = go.Scatter(
        x = filter_coefs[0, :], y = filter_coefs[1, :],
        mode = 'markers',
        marker = dict(
            size = 10,
            color = '#bbbbbb',
            opacity = 0.4
        )
    )

    plots.append(scatter_all)

    for i in range(n_coreset):
        start = i * n_coreset
        end = (i + 1) * n_coreset
        scatter_coreset = go.Scatter(
            x = coreset[0, start : end], y = coreset[1, start : end],
            mode = 'markers',
            marker = dict(
                size = 8,
                color = 1.0,
                colorscale = colors[0],
                opacity = 1.
            )
        )

        plots.append(scatter_coreset)
        
    layout = go.Layout(
        margin = dict(l=0, r=0, b=0, t=0),
        xaxis_title = 'basis_0',
        yaxis_title = 'basis_1',
        showlegend = False
    )
else:
    scatter_all = go.Scatter3d(
        x = filter_coefs[0,:], y = filter_coefs[1,:], z = filter_coefs[2,:],
        mode = 'markers',
        name = 'all data points',
        marker = dict(
            size = 3,
            color = '#bbbbbb',
            opacity = 0.3
        )
    )

    plots.append(scatter_all)

    scatter_coreset = go.Scatter3d(
        x = coreset[0,:], y = coreset[1,:], z = coreset[2,:],
        mode = 'markers',
        marker = dict(
            size = 8,
            color = coreset[3, :],
            colorbar = dict(
                title = '4th dimension'
            ),
            colorscale = colors[0],
            opacity = 1.
        )
    )

    plots.append(scatter_coreset)

    # for i in range(n_meas):
    #     start = i * n_coreset
    #     end = (i + 1) * n_coreset

    #     scatter_coreset = go.Scatter3d(
    #         x = coreset[0, start : end], y = coreset[1, start : end], z = coreset[2, start : end],
    #         mode = 'markers',
    #         marker = dict(
    #             size = 8,
    #             color = coreset[3, start : end],
    #             colorbar = dict(
    #                 title = 'meas set ' + str(i + 1),
    #                 x = 1 + .1 * i
    #             ),
    #             colorscale = colors[i],
    #             opacity = 1.
    #         )
    #     )

    #     plots.append(scatter_coreset)

    layout = go.Layout(
        title = 'Optimal filter set',
        scene_xaxis_title = 'basis_0',
        scene_yaxis_title = 'basis_1',
        scene_zaxis_title = 'basis_2',
        showlegend = False
    )

fig = go.FigureWidget(data = plots, layout = layout)
fig

FigureWidget({
    'data': [{'marker': {'color': '#bbbbbb', 'opacity': 0.4, 'size': 10},
              'mode': 'markers',
              'type': 'scatter',
              'uid': 'db8466b3-0871-4ab8-aff6-b4da1b230fe4',
              'x': array([4.1062093, 5.272908 , 1.4850237, ..., 5.0671625, 4.1337786, 3.7598524],
                         dtype=float32),
              'y': array([-0.2923655 ,  0.45714274,  0.5475849 , ..., -0.46964395,  1.1053765 ,
                          -0.53185517], dtype=float32)},
             {'marker': {'color': 1.0,
                         'colorscale': [[0.0, 'rgb(176, 242, 188)'],
                                        [0.16666666666666666, 'rgb(137, 232,
                                        172)'], [0.3333333333333333, 'rgb(103, 219,
                                        165)'], [0.5, 'rgb(76, 200, 163)'],
                                        [0.6666666666666666, 'rgb(56, 178, 163)'],
                                        [0.8333333333333334, 'rg

In [24]:
print(opt_filters)

[Config(alphas = [ 7.84742131 96.99075763], theta = 146.8124952198959, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [179.03722173 174.09347268], theta = 176.71535505733206, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [178.51361608 133.82906752], theta = 89.32628640778263, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [138.35504924 135.30851471], theta = 87.64681225053934, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [137.99852254 147.15837265], theta = 6.286870742298339, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Config(alphas = [155.93308026  47.29240669], theta = 39.97049140372845, waveplates = [Waveplate(th = 78740.0, bf = 0.00936), Waveplate(th = 66040.0, bf = 0.00872)])
 Conf